# RAG application for "A Guide for First-Time Parents"

## Install tika and parse PDF file

- Install libraries
- Download PDF from the website [The Asian Parent](https://th.theasianparent.com/%E0%B8%84%E0%B8%B9%E0%B9%88%E0%B8%A1%E0%B8%B7%E0%B8%AD%E0%B8%94%E0%B8%B9%E0%B9%81%E0%B8%A5%E0%B8%A5%E0%B8%B9%E0%B8%81)
- Parse a PDF file using `tika`
- Clean text (using a simple created function)

In [17]:
%%capture
!pip install tika
!pip install unidecode
!pip install openai
!pip install faiss-cpu==1.7.4

In [18]:
import pandas as pd
import numpy as np
import faiss

import tika
tika.initVM()
from tika import parser
from unidecode import unidecode

In [19]:
parsed_book = parser.from_file("baby_0_3.pdf")

n_pages = int(parsed_book["metadata"]["xmpTPg:NPages"])
print(n_pages)

86


In [20]:
def clean_text(text: str):
    """Clean parsed text from PDF for embedding"""
    text = text.replace("\uf70a", "่")
    text = text.replace("�ำ", "ำ")
    text = text.replace("�า", "ำ")
    return text

In [21]:
content = parsed_book["content"]
content_processed = clean_text(content)
pages = content_processed.split("\n\n\n\n")

In [22]:
pages_strip = [" ".join(page.split()) for page in pages]  # strip extra spaces from page

## Perform RAG for each page in the book

- As we skim through, each page already contains a single content
- Chunk information to default `chunk_size` of 2048

In [23]:
def convert_page_to_chunk(page_text, chunk_size: int = 2048):
    chunks = [page_text[i:i + chunk_size] for i in range(0, len(page_text), chunk_size)]
    return chunks

In [24]:
chunks = []
for text in pages_strip:
    chunks.extend(convert_page_to_chunk(text))

In [25]:
len(chunks)

101

In [35]:
import json

with open("baby_0_3.json", "w") as f:
    json.dump(chunks, f, indent=4, ensure_ascii=False)

## Prompting using RAG

- Embed text chunks with and store using `faiss`
- Embed query using the same embedding script
- Find the closest text chunks
- Add information and perform RAG

In [26]:
import os
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

def get_embedding(text: str, model: str = "text-embedding-3-small"):
    """Function to perform text embedding, see options from OpenAI's website at

    https://platform.openai.com/docs/guides/embeddings/embedding-models
    """
    text = text.replace("\n", " ")
    return client.embeddings.create(input = [text], model=model).data[0].embedding

In [27]:
text_chunks = pd.DataFrame(chunks, columns=["text"])["text"]
text_embeddings = np.vstack(text_chunks.apply(get_embedding))

In [28]:
print(text_embeddings.shape) # shape of text embedding

(101, 1536)


In [29]:
d = text_embeddings.shape[1]
index = faiss.IndexFlatL2(d)
index.add(text_embeddings)

In [30]:
import pickle
with open("faiss_index.pkl", "wb") as f:
    pickle.dump(index, f)

In [31]:
question = "ถ้าลูกดูดนิ้วต้องทำอย่างไรบ้าง"
# question = "ถ้าลูกดูดมือต้องทำอย่างไรบ้าง" # prompt augmentation???
question_embeddings = np.array([get_embedding(question)])

D, I = index.search(question_embeddings, k=2)
print(I)

retrieved_chunk = [chunks[i] for i in I.tolist()[0]]

[[95 26]]


In [33]:
retrieved_chunk[0]

'82 ดูดนิ้ว การดดูนิว้เป็นพฤตกิรรมท่ีพบได้บ่อยและเกดิขึน้ได้ตามปกตขิองพฒันาการเดก็ เดก็เริม่ดดูนิว้ตัง้แต่ อยู่ในครรภ์มารดาจนถึง 2 ขวบ และมักจะค่อยๆ เลิกดูดไปเอง เด็กอาจดูดนิ้วมากขึ้นในบางภาวะ ได้แก่ เมื่อเครียด ง่วงนอน กังวล กลัว ถูกขัดใจ เหนื่อย หรือเวลาเพลินๆ เช่น ดูทีวี โดยทั่วไปแล้วการติดดูดนิ้วนี้ มักจะหายไปก่อนอายุ 5 ปี สาเหตุที่เด็กชอบดูดนิ้ว 1. เป็นพฤติกรรมตามธรรมชาติ ซึ่งพบบ่อยในระยะ 1 ขวบปีแรก 2. ถูกปล่อยให้ยึดติดกับพฤติกรรมน้ี โดยไม่ได้รับการฝึกฝนหรือเบี่ยงเบนแก้ไข จนเด็กติดกลาย เป็นนิสัย ไม่ยอมเลิกดูดนิ้วเมื่อโตขึ้น 3. เป็นวธีิการระบายความเครียดจากความวติกกงัวล เช่น การพลดัพรากจากพ่อแม่ ตืน่เต้น เป็นต้น 4. การเลี้ยงดูที่ไม่เหมาะสม เช่น ถูกทอดทิ้ง ขาดการกระตุ้น เป็นต้น วิธีการแก้ไข แม้ว่าการดูดนิ้วจะเป็นพฤติกรรมตามธรรมชาติ แต่จำเป็นต้องปรับลดให้หมดไปก่อนโต วิธีการที่ ใช้ได้ผล เช่น 1. ควรให้ความสนใจเด็ก โดยการเล่น หยอกล้อกับเด็ก เพื่อไม่ให้เด็กกระตุ้นตัวเองตามลำพัง 2. เบี่ยงเบนความสนใจ โดยใช้ของเล่นที่ต้องใช้มือหรือให้เด็กทำกิจกรรมอื่นๆ ที่ต้องใช้มือแทน 3. ให้เดก็ออกกำลงักาย

Compare the prompt with (RAG) and without retrieved information

In [23]:
def complete_text(prompt: str):
    response = client.chat.completions.create(
      model="gpt-3.5-turbo-0125",
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to generate output prompt for parent who are asking questions about newborn in Thai."},
        {"role": "user", "content": prompt}
      ]
    )
    return response.choices[0].message.content

In [24]:
prompt_with_context = f"""
Context information is below.
---------------------
{retrieved_chunk}
---------------------
Given the context information and not prior knowledge, answer the given query in Thai. The answer should be concise and clear.
Query: {question}
Answer:
"""

prompt_no_context = f"""
Answer the given query in Thai. The answer should be concise and clear.
Query: {question}
Answer:
"""

In [25]:
output_no_rag = complete_text(prompt_no_context)
print(output_no_rag)

การให้ลูกดูดนิ้วเป็นเรื่องปกติ ไม่จำเป็นต้องทำอะไรเพิ่มเติม สามารถให้ลูกดูดนิ้วได้เพราะเป็นการทำให้ลูกรู้สึกถูกประทับใจและปลอดภัย แต่ควรตรวจสอบว่านิ้วมือสะอาดเสมอ และเก็บแผ่นเคราและรัดฝาตีนเท้าให้ลูกได้นิดๆ บ้างเพื่อป้องกันการกระแทกกระทบผิวของลูกได้บ้าง


In [26]:
output_rag = complete_text(prompt_with_context)
print(output_rag)

การดูดนิ้วเป็นพฤติกรรมที่พบได้ในเด็ก โดยมักจะหายไปเองก่อนอายุ 5 ปี วิธีการช่วยลูกเลิกดูดนิ้วได้รวมถึงการเล่นกับเด็กเพื่อไม่ให้กระตุ้นตัวเอง, เบี่ยงเบนความสนใจไปที่กิจกรรมอื่น, และช่วยลูกระบายความเครียดด้วยวิธีการต่างๆ คุณสามารถปรับลดพฤติกรรมดูดนิ้วได้ด้วยวิธีดังกล่าว.
